### Input files: Price-Today, orders-log
### Output files: Price-for-Order, orders-log, monthly-sales

In [2]:
import glob
import os
import pandas as pd
from datetime import date, timedelta, datetime
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}', 'target':'{:,.2f}',
              'cost':'{:,.2f}','price':'{:,.2f}','%change':'{:,.2f}','%profit':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'cst_amt':'{:,.2f}','mkt_amt':'{:,.2f}','difa':'{:,.2f}','current':'{:,.2f}','change':'{:,.2f}',  
              'pct':'{:,.2f}%','net':'{:,.2f}','%change':'{:,.2f}%','%profit':'{:,.2f}%'
              }
pd.options.display.max_rows = 50
pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-10-24 2025-10-23


In [3]:
num_business_days = BDay(2)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-10-24
yesterday: 2025-10-22


In [4]:
# If run after actual work day
#today = today - timedelta(days=2)
#print(today)

In [5]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-10-24 20:19:34


### Restart and Run All Cells

In [7]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\A5\Ruby
ruby_path = os.path.join(base_path, "Ruby")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [8]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Ruby path (ruby_path): {ruby_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Ruby path (ruby_path): C:\Users\PC1\OneDrive\A5\Ruby
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


## Must rerun whenever there is change in orders

In [10]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(18, 8)

In [11]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [12]:
prices = pd.read_csv(input_file)
prices.shape

(168, 10)

In [13]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(18, 17)

In [14]:
df_merge['spd'] = 0

In [15]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(18, 12)

In [16]:
file_name = "Price-for-Order.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [17]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\Price-for-Order.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-for-Order.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-for-Order.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-for-Order.csv


In [18]:
df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(god_file, header=True, index=False)
df_merge[colo].to_csv(icd_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [20]:
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [21]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [22]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,AIMIRT,-21,RND15pct,SET,2500,9.1,10.3,-0.1,-0.961538,0,2025-08-19
B,CPF,17,,SET50,10000,23.4,21.7,-0.3,-1.363636,1,2025-08-29
B,GVREIT,-10,RND20Pct,SET,6000,6.0,6.5,0.05,0.775194,0,2025-08-27
B,IVL,-4,CSSTGT,SET50,1800,20.6,21.0,0.3,1.449275,1,2025-08-27
B,JMART,-2,RD80Pct,SET50,3200,8.0,8.1,-0.2,-2.409639,1,2025-05-09
B,JMT,10,RD75pct,SET50,2800,11.0,10.0,-0.3,-2.912621,1,2025-08-26
B,NER,-7,CSSMIN,SET,5000,4.0,4.14,0.02,0.485437,1,2025-08-22
B,ORI,-15,RD70Pct,SET100,10000,2.0,2.3,-0.06,-2.542373,0,2025-03-13
B,RCL,1,RD35pct,SET100,3000,26.0,25.75,-0.25,-0.961538,1,2025-08-22
B,TFFIF,-1,RD30Pct,SET,10000,6.0,6.05,-0.05,-0.819672,1,2025-08-28
B,WHAIR,-23,RD50pct,SET,10000,4.9,5.95,-0.05,-0.833333,0,2025-08-18
S,DIF,0,RD40Pct,SET,10000,9.3,9.3,0.05,0.540541,1,2025-08-15
S,MCS,10,20Pct,SET,6000,8.75,8.25,0.05,0.609756,1,2025-08-26
S,PTG,44,COS,SET100,3600,11.4,8.5,-0.25,-2.857143,0,2025-03-10
S,PTT,14,,SET50,25

In [23]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [24]:
file_name   = 'orders-log.csv'
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\orders-log.csv


In [25]:
df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3').style.format(format_dict).hide(axis="index")

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
B,JMART,-2,RD80Pct,SET50,"3,200",8.00,8.10,-0.20,-2.41%,1,2025-05-09,"25,600.00"
B,RCL,1,RD35pct,SET100,"3,000",26.00,25.75,-0.25,-0.96%,1,2025-08-22,"78,000.00"
B,TFFIF,-1,RD30Pct,SET,"10,000",6.00,6.05,-0.05,-0.82%,1,2025-08-28,"60,000.00"
S,DIF,0,RD40Pct,SET,"10,000",9.30,9.30,0.05,0.54%,1,2025-08-15,"93,000.00"
S,WHART,1,RD30Pct,SET,"5,000",9.70,9.65,-0.05,-0.52%,1,2025-08-18,"48,500.00"


In [26]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [27]:
df.query('active == 1').style.format(format_dict).hide(axis="index")

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
B,CPF,17,nan,SET50,"10,000",23.40,21.70,-0.30,-1.36%,1,2025-08-29,"234,000.00"
B,IVL,-4,CSSTGT,SET50,"1,800",20.60,21.00,0.30,1.45%,1,2025-08-27,"37,080.00"
B,JMART,-2,RD80Pct,SET50,"3,200",8.00,8.10,-0.20,-2.41%,1,2025-05-09,"25,600.00"
B,JMT,10,RD75pct,SET50,"2,800",11.00,10.00,-0.30,-2.91%,1,2025-08-26,"30,800.00"
B,NER,-7,CSSMIN,SET,"5,000",4.00,4.14,0.02,0.49%,1,2025-08-22,"20,000.00"
B,RCL,1,RD35pct,SET100,"3,000",26.00,25.75,-0.25,-0.96%,1,2025-08-22,"78,000.00"
B,TFFIF,-1,RD30Pct,SET,"10,000",6.00,6.05,-0.05,-0.82%,1,2025-08-28,"60,000.00"
S,DIF,0,RD40Pct,SET,"10,000",9.30,9.30,0.05,0.54%,1,2025-08-15,"93,000.00"
S,MCS,10,20Pct,SET,"6,000",8.75,8.25,0.05,0.61%,1,2025-08-26,"52,500.00"
S,TVO,12,DTD,SET,"4,000",25.50,24.00,-0.10,-0.41%,1,2025-08-22,"102,000.00"


In [28]:
df.query('active == 0').style.format(format_dict).hide(axis="index")

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
B,AIMIRT,-21,RND15pct,SET,"2,500",9.10,10.30,-0.10,-0.96%,0,2025-08-19,"22,750.00"
B,GVREIT,-10,RND20Pct,SET,"6,000",6.00,6.50,0.05,0.78%,0,2025-08-27,"36,000.00"
B,ORI,-15,RD70Pct,SET100,"10,000",2.00,2.30,-0.06,-2.54%,0,2025-03-13,"20,000.00"
B,WHAIR,-23,RD50pct,SET,"10,000",4.90,5.95,-0.05,-0.83%,0,2025-08-18,"49,000.00"
S,PTG,44,COS,SET100,"3,600",11.40,8.50,-0.25,-2.86%,0,2025-03-10,"41,040.00"
S,PTT,14,nan,SET50,"2,500",35.25,31.75,0.75,2.42%,0,2025-09-01,"88,125.00"
S,SYNEX,34,RD70pct,SET,"2,500",13.90,10.50,-0.10,-0.94%,0,2025-08-26,"34,750.00"


In [29]:
df.query('spd <= -40 and trade == "B"').style.format(format_dict).hide(axis="index")

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [30]:
df.query('trade == "B" & spd > 0').style.format(format_dict).hide(axis="index")

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
B,CPF,17,nan,SET50,"10,000",23.40,21.70,-0.30,-1.36%,1,2025-08-29,"234,000.00"
B,JMT,10,RD75pct,SET50,"2,800",11.00,10.00,-0.30,-2.91%,1,2025-08-26,"30,800.00"
B,RCL,1,RD35pct,SET100,"3,000",26.00,25.75,-0.25,-0.96%,1,2025-08-22,"78,000.00"


In [31]:
df.query('trade == "S" & spd < 0').style.format(format_dict).hide(axis="index")

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [32]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

In [33]:
file_name = "orders-log.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [34]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\orders-log.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\orders-log.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\orders-log.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\orders-log.csv


In [35]:
df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(god_file, header=True, index=False)
df[colx].to_csv(icd_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [37]:
df['percent'] = df['%change']

In [38]:
df.query('percent < -2').style.format(format_dict).hide(axis="index")

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
B,JMART,-2,RD80Pct,SET50,"3,200",8.00,8.10,-0.20,-2.41%,1,2025-05-09,"25,600.00",-2.409639
B,JMT,10,RD75pct,SET50,"2,800",11.00,10.00,-0.30,-2.91%,1,2025-08-26,"30,800.00",-2.912621
B,ORI,-15,RD70Pct,SET100,"10,000",2.00,2.30,-0.06,-2.54%,0,2025-03-13,"20,000.00",-2.542373
S,PTG,44,COS,SET100,"3,600",11.40,8.50,-0.25,-2.86%,0,2025-03-10,"41,040.00",-2.857143


In [39]:
df.query('percent >= 2.00').style.format(format_dict).hide(axis="index")

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
S,PTT,14,nan,SET50,"2,500",35.25,31.75,0.75,2.42%,0,2025-09-01,"88,125.00",2.419355


In [40]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False).style.format(format_dict).hide(axis="index")

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
B,CPF,17,nan,SET50,"10,000",23.40,21.70,-0.30,-1.36%,1,2025-08-29,"234,000.00",-1.363636
B,JMT,10,RD75pct,SET50,"2,800",11.00,10.00,-0.30,-2.91%,1,2025-08-26,"30,800.00",-2.912621
B,RCL,1,RD35pct,SET100,"3,000",26.00,25.75,-0.25,-0.96%,1,2025-08-22,"78,000.00",-0.961538
B,TFFIF,-1,RD30Pct,SET,"10,000",6.00,6.05,-0.05,-0.82%,1,2025-08-28,"60,000.00",-0.819672
B,JMART,-2,RD80Pct,SET50,"3,200",8.00,8.10,-0.20,-2.41%,1,2025-05-09,"25,600.00",-2.409639
B,IVL,-4,CSSTGT,SET50,"1,800",20.60,21.00,0.30,1.45%,1,2025-08-27,"37,080.00",1.449275
B,NER,-7,CSSMIN,SET,"5,000",4.00,4.14,0.02,0.49%,1,2025-08-22,"20,000.00",0.485437
B,GVREIT,-10,RND20Pct,SET,"6,000",6.00,6.50,0.05,0.78%,0,2025-08-27,"36,000.00",0.775194
B,ORI,-15,RD70Pct,SET100,"10,000",2.00,2.30,-0.06,-2.54%,0,2025-03-13,"20,000.00",-2.542373
B,AIMIRT,-21,RND15pct,SET,"2,500",9.10,10.30,-0.10,-0.96%,0,2025-08-19,"22,750.00",-0.961538


### Sell recently purchased stocks if profitable

In [42]:
#pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(113, 4)

In [43]:
colu = 'name date_x qty cost price target %change %profit cst_amt mkt_amt difa'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [44]:
buys_price['cst_amt'] = buys_price['qty'] * buys_price['cost']
buys_price['mkt_amt'] = buys_price['qty'] * buys_price['price']
buys_price['target'] = buys_price['cost'] * 1.1

In [45]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(108, 11)

In [46]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('%profit',ascending=False).style.format(format_dict).hide(axis="index")

name,date_x,qty,cost,price,target,%change,%profit,cst_amt,mkt_amt,difa
DIF,2025-08-14,"10,000",8.45,9.30,9.29,0.54%,10.06%,"84,500.00","93,000.00","8,500.00"
AIMIRT,2025-08-13,"2,500",9.80,10.30,10.78,-0.96%,5.10%,"24,500.00","25,750.00","1,250.00"
AIMIRT,2025-08-08,"2,500",9.90,10.30,10.89,-0.96%,4.04%,"24,750.00","25,750.00","1,000.00"
PTT,2025-09-01,"2,500",31.00,31.75,34.10,2.42%,2.42%,"77,500.00","79,375.00","1,875.00"
DIF,2024-11-08,"5,000",9.10,9.30,10.01,0.54%,2.20%,"45,500.00","46,500.00","1,000.00"


In [47]:
current_path = os.getcwd()
base_path = os.path.dirname(current_path)
ruby_path = os.path.join(base_path, "Ruby")
print(f"Ruby path (ruby_path): {ruby_path}") 

Ruby path (ruby_path): C:\Users\PC1\OneDrive\A5\Ruby


In [48]:
file_name = "watch-list-sell.xlsx"
output_file = os.path.join(ruby_path, file_name)
watch_list[colu].sort_values('%profit',ascending=False).to_excel(output_file, header=True, index=False)

In [49]:
float_formatter = "฿{:,.2f}"
float_formatter.format(watch_list.difa.sum())

'฿13,625.00'

In [50]:
sell_list = buys_price.loc[buys_price['%profit'] >= 10.00,colu]
sell_list.sort_values('%profit',ascending=False).style.format(format_dict).hide(axis="index")

name,date_x,qty,cost,price,target,%change,%profit,cst_amt,mkt_amt,difa
DIF,2025-08-14,"10,000",8.45,9.30,9.29,0.54%,10.06%,"84,500.00","93,000.00","8,500.00"


In [51]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sell_list.cst_amt.sum())

'฿84,500.00'

In [52]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sell_list.mkt_amt.sum())

'฿93,000.00'

In [53]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sell_list.difa.sum())

'฿8,500.00'

### Profit per Month

In [55]:
year = 2025
month = 10
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2025 AND MONTH(sells.date) = 10
ORDER BY sells.date DESC, name


In [56]:
sells_df.style.format(format_dict).hide(axis="index")

name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
PTT,2025,10,31.25,31.00,0.25,"2,500","78,125.00","77,500.00",625.00,0.81%,8
RCL,2025,10,26.50,26.00,0.50,"3,000","79,500.00","78,000.00","1,500.00",1.92%,1
ORI,2025,10,2.72,2.62,0.10,"10,000","27,200.00","26,200.00","1,000.00",3.82%,23


In [57]:
print(f"Monthly Profit Amount: {sells_df.gross.sum():,.2f}")

Monthly Profit Amount: 3,125.00


In [58]:
file_name = "monthly-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [59]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\monthly-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\monthly-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\monthly-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\monthly-sales.csv


In [60]:
sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(god_file, header=True, index=False)
sells_df.to_csv(icd_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [61]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict).hide(axis="index")

name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day
PTT,2025,10,31.25,31.00,0.25,"2,500","78,125.00","77,500.00",625.00,0.81%,8,0.100806
RCL,2025,10,26.50,26.00,0.50,"3,000","79,500.00","78,000.00","1,500.00",1.92%,1,1.923077
ORI,2025,10,2.72,2.62,0.10,"10,000","27,200.00","26,200.00","1,000.00",3.82%,23,0.165948


In [62]:
float_formatter = "฿{:,.2f}"
float_formatter.format(sells_df.gross.sum())

'฿3,125.00'

In [63]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: 0.73%


In [64]:
#dt = date(2023, 11, 8)
dt = today
dt.strftime("%B %d, %Y")

'October 24, 2025'

In [65]:
print("Hello, {}! Today is {:%B %d, %Y}.".format("Alice", dt))

Hello, Alice! Today is October 24, 2025.


In [66]:
conlite.commit()
conlite.close()

In [67]:
# Get the current time
current_time = datetime.now()
# Format the time to remove millisAeconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-10-24 20:19:35
